<a href="https://colab.research.google.com/github/devdaveddev/BOTDb/blob/main/DBbot29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import openai
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
from textblob import TextBlob
import torch
import random
import numpy as np

# Initialize NLP models
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
summarizer = pipeline("summarization")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score


openai.api_key = "YOUR_OPENAI_API_KEY"


def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    response = requests.get(url).json()
    return response.get("extract", "No information found.")


def fetch_reddit_data(topic):
    headers = {"User-Agent": "AI-Debater"}
    url = f"https://www.reddit.com/r/debate/comments.json?q={topic}&sort=top"
    response = requests.get(url, headers=headers).json()
    posts = [post['data']['title'] for post in response['data']['children'][:5]]
    return posts


def fetch_reddit_cmv_data(topic):
    headers = {"User-Agent": "AI-Debater"}
    url = f"https://www.reddit.com/r/changemyview/comments.json?q={topic}&sort=top"
    response = requests.get(url, headers=headers).json()
    posts = [post['data']['title'] for post in response['data']['children'][:5]]
    return posts


def select_model():
    return "GPT-4"  # Alternatively, use Llama 2 or Mixtral


def validate_with_knowledge_graph(topic):
    return f"Validated factual information on {topic}."

#Chain-of-Thought reasoning
def generate_human_like_arguments(topic, debate_type):
    wikipedia_info = fetch_wikipedia_data(topic)
    reddit_info = fetch_reddit_data(topic)
    reddit_cmv_info = fetch_reddit_cmv_data(topic)
    fact_check = validate_with_knowledge_graph(topic)

    weights = np.array([
        [0.7, 0.1, 0.1, 0.1],  # Educated: More Wikipedia, less Reddit, AI
        [0.0, 0.0, 1.0, 0.0],  # Radical: Only AIs
        [0.2, 0.5, 0.2, 0.1]   # Casual: More Reddit, some Wikipedia, little AI
    ])
    weight_labels = {"educated": 0, "radical": 1, "casual": 2}
    w_wiki, w_reddit, w_reddit_cmv, w_ai = weights[weight_labels.get(debate_type, 0)]

    combined_context = (
        f"Step 1: Gathering reliable sources.\n"
        f"Wikipedia Info: {wikipedia_info * w_wiki}\n\n"
        f"Step 2: Analyzing user discussions.\n"
        f"Reddit Insights: {reddit_info * w_reddit}\n\n"
        f"Step 3: Evaluating counterarguments from Change My View.\n"
        f"Change My View Insights: {reddit_cmv_info * w_reddit_cmv}\n\n"
        f"Step 4: Fact-checking for validity.\n"
        f"Fact Check: {fact_check}"
    )

    response = openai.ChatCompletion.create(
        model=select_model(),
        temperature=1.2,
        messages=[
            {"role": "system", "content": "You generate arguments by merging factual information from Wikipedia, Reddit, Change My View discussions, knowledge graphs, and OpenAI-generated insights, ensuring a human-like debate experience with step-by-step logical reasoning."},
            {"role": "user", "content": f"Debate this topic with strong reasoning: {topic}\n\n{combined_context}"}
        ]
    )
    return response["choices"][0]["message"]["content"]


def analyze_bias(user_input):
    sentiment_score = bert_sentiment_analysis(user_input)
    if sentiment_score >= 4:
        return "Highly Positive (Potential Bias)"
    elif sentiment_score <= 1:
        return "Highly Negative (Potential Bias)"
    return "Neutral"


def ai_debate():
    debate_type = input("Choose debate style (educated, radical, casual): ")
    topic = input("Enter a topic to debate: ")

    print("\nFetching arguments...\n")

    while True:
        ai_argument = generate_human_like_arguments(topic, debate_type)
        print("\nAI's Argument:")
        print(ai_argument)

        print("\nEnter your counter-argument or type 'exit' to stop:")
        user_argument = input()

        if user_argument.lower() == 'exit':
            print("Debate session ended. Thanks for debating!")
            break

        bias_analysis = analyze_bias(user_argument)
        print("\nBias Analysis:", bias_analysis)

        topic = f"Counter to: {user_argument}"

if __name__ == "__main__":
    ai_debate()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import requests
import llama_cpp
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
from textblob import TextBlob
import torch
import random
import numpy as np
import os
import tensorflow as tf

# Initialize NLP models
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
summarizer = pipeline("summarization")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score

llama_cpp.api_key = os.getenv("LLAMA_API_KEY")


class DebateRLModel:
    def __init__(self):
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1, activation='linear')
        ])
        self.model.compile(optimizer='adam', loss='mse')
        self.memory = []

    def train(self, X, y):
        self.model.fit(np.array(X), np.array(y), epochs=10, verbose=0)

    def predict(self, X):
        return self.model.predict(np.array(X))

    def remember(self, state, reward):
        self.memory.append((state, reward))

    def replay(self):
        if len(self.memory) > 10:
            batch = random.sample(self.memory, 10)
            X, y = zip(*batch)
            self.train(X, y)

rl_model = DebateRLModel()

def fetch_data(url, headers=None):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Data fetch error: {e}")
        return {}

def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    return fetch_data(url).get("extract", "No information found.")

def fetch_stackexchange_data(topic):
    url = f"https://api.stackexchange.com/2.3/search?order=desc&sort=relevance&intitle={topic}&site=history"
    response = fetch_data(url)
    return [item["title"] for item in response.get("items", [])[:5]]

def fetch_reddit_data(topic):
    url = f"https://www.reddit.com/r/debaterelated/search.json?q={topic}&restrict_sr=1"
    headers = {'User-Agent': 'debate-bot'}
    response = fetch_data(url, headers)
    return [post["data"]["title"] for post in response.get("data", {}).get("children", [])[:5]]

def fetch_philosopher_data():
    philosophers = ["Socrates", "Plato", "Nietzsche", "Franz Kafka", "Neil deGrasse Tyson", "Alex O'Connor"]
    return {philosopher: fetch_wikipedia_data(philosopher) for philosopher in philosophers}

def validate_with_knowledge_graph(topic):
    return f"Validated factual information on {topic}."

def weighted_text(text, weight):
    return text[:int(len(text) * weight)] if isinstance(text, str) else text[:max(1, int(len(text) * weight))]

def update_weights(weights, feedback_score, learning_rate=0.1):
    new_weights = weights + learning_rate * (feedback_score - 3) * weights
    rl_model.remember(weights, new_weights)
    rl_model.replay()
    return new_weights

def assess_debate_response(response):
    criteria = ["Premise", "Propositions", "Arguments", "Rebuttals", "Evidence"]
    scores = {criterion: int(input(f"Rate the response on {criterion} (1-5): ")) for criterion in criteria}
    return scores

def generate_human_like_arguments(topic, debate_type, role, weights):
    wikipedia_info = fetch_wikipedia_data(topic)
    stackexchange_info = fetch_stackexchange_data(topic)
    reddit_info = fetch_reddit_data(topic)
    philosopher_data = fetch_philosopher_data()
    fact_check = validate_with_knowledge_graph(topic)

    weight_labels = {"educated": 0, "radical": 1, "casual": 2}
    w_wiki, w_stack, w_reddit, w_ai = weights[weight_labels.get(debate_type, 0)]

    combined_context = (
        f"Wikipedia Info: {weighted_text(wikipedia_info, w_wiki)}\n"
        f"StackExchange Insights: {weighted_text(' '.join(stackexchange_info), w_stack)}\n"
        f"Reddit Insights: {weighted_text(' '.join(reddit_info), w_reddit)}\n"
        f"Fact Check: {fact_check}\n"
        f"Philosopher Insights: {weighted_text(str(philosopher_data), w_ai)}"
    )

    response = llama_cpp.ChatCompletion.create(
        model="llama-cpp-python",
        temperature=1.2,
        messages=[
            {"role": "system", "content": f"You are roleplaying as {role}. Follow structured reasoning."},
            {"role": "user", "content": f"Debate this topic: {topic}\n{combined_context}"}
        ]
    )
    return response["choices"][0]["message"]["content"], weights

def ai_debate():
    topic = input("Enter a topic to debate: ")
    debate_type = input("Choose debate style (educated, radical, casual): ")
    role = input("Choose AI roleplay style: ")

    weights = np.array([
        [0.7, 0.2, 0.1, 0.1],
        [0.0, 1.0, 0.0, 0.0],
        [0.2, 0.5, 0.3, 0.2]
    ])

    while True:
        ai_argument, weights = generate_human_like_arguments(topic, debate_type, role, weights)
        print("\nAI's Argument:", ai_argument)
        user_argument = input("Enter your counter-argument or 'exit': ")
        if user_argument.lower() == 'exit': break
        feedback_scores = assess_debate_response(user_argument)
        feedback_score = int(input("Rate AI's argument (1-5): "))
        weights = update_weights(weights, feedback_score)
        topic = f"Counter to: {user_argument}"
